# generating a large random network topology

In [1]:
using Random
# naive approach: 
function gentopo(n,k,seed)
    p = k/(n-1)
    rng = MersenneTwister(seed);
    rand(rng,n,n) .< p
end

gentopo (generic function with 1 method)

In [2]:
n = 50 # number of neurons
k = 10 # number of synapses per neuron
seed = 1
A = gentopo(n,k,seed)

50×50 BitArray{2}:
 false  false   true  false  false  …   true  false   true  false   true
 false  false  false  false  false      true  false  false  false  false
 false  false  false   true  false     false  false  false  false   true
  true  false  false  false  false     false   true  false   true  false
 false   true   true  false  false     false  false  false   true   true
 false  false  false  false  false  …  false  false  false   true  false
 false   true  false   true   true     false   true   true  false  false
 false  false  false  false  false     false  false   true   true  false
 false  false  false  false   true     false  false  false  false   true
 false  false  false  false  false     false  false   true   true  false
 false  false  false  false  false  …  false  false   true  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false   true  false  false      true  false  false  false  false
     ⋮                          

In [4]:
j = 17 # example spiking neuron
post = A[:,j]

50-element BitArray{1}:
 false
 false
 false
  true
 false
  true
 false
 false
 false
  true
 false
  true
 false
     ⋮
 false
 false
 false
 false
  true
 false
 false
 false
 false
 false
 false
  true

 ### using sparse matrices

In [5]:
using SparseArrays
# needs less memory (n*k instead of n^2)
function gensparsetopo(n,k,seed)
    rng = MersenneTwister(seed);
    p = k/(n-1)
    A = sprand(Bool,n,n,p)
end

gensparsetopo (generic function with 1 method)

In [6]:
Asparse = gensparsetopo(n,k,seed)

50×50 SparseMatrixCSC{Bool,Int64} with 527 stored entries:
  [5 ,  1]  =  true
  [25,  1]  =  true
  [30,  1]  =  true
  [31,  1]  =  true
  [32,  1]  =  true
  [34,  1]  =  true
  [48,  1]  =  true
  [49,  1]  =  true
  [7 ,  2]  =  true
  [10,  2]  =  true
  [12,  2]  =  true
  [14,  2]  =  true
  ⋮
  [41, 49]  =  true
  [50, 49]  =  true
  [1 , 50]  =  true
  [4 , 50]  =  true
  [5 , 50]  =  true
  [7 , 50]  =  true
  [9 , 50]  =  true
  [16, 50]  =  true
  [28, 50]  =  true
  [37, 50]  =  true
  [46, 50]  =  true
  [50, 50]  =  true

In [7]:
j = 17 # example spiking neuron
post = A[:,j]

50-element BitArray{1}:
 false
 false
 false
  true
 false
  true
 false
 false
 false
  true
 false
  true
 false
     ⋮
 false
 false
 false
 false
  true
 false
 false
 false
 false
 false
 false
  true

### generating network topology on the fly

In [8]:
# no need to store network topology of size n*k
using StatsBase
j = 18
mt_rng = MersenneTwister(j)
post = sample(1:n-1,k) # generate postsynaptic neuron index

10-element Array{Int64,1}:
 32
 43
 26
 24
 15
  8
 42
 49
  2
 12

###### bottleneck: seeding the random number generator

In [9]:
using BenchmarkTools
mt = MersenneTwister(seed)
@btime Random.seed!(mt,1);

  11.705 μs (2 allocations: 112 bytes)


###### Solution: using faster random number generator

In [10]:
using RandomNumbers.Xorshifts
r = Xoroshiro128Star(seed)
@btime Random.seed!(r,1);

  18.879 ns (0 allocations: 0 bytes)
